In [ ]:
#Useful imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
def matshow(A, title=''):
    plt.matshow(A)
    plt.axis('off')
    plt.title(title)

## A.  Simple nonblind deconvolution

In [ ]:
k = np.array([-1, 1]) # convolution kernel

s = 4
S = np.array([s, s])

# Matlab analogy
N = np.array(
    [[16,     2,     3,   13],
    [5 ,   11 ,   10 ,    8],
    [9 ,    7 ,    6 ,   12],
    [4 ,   14 ,   15 ,    1]])

# Create Image with random entries between 0 and 1
#N = np.random.random(size = S) 

# create zero-padded and mirrored version of k.
matshow(N, '$N$')

### 1. Standard convolution

In [ ]:
from scipy.signal import convolve2d
k_2D = k[::-1].reshape(1, -1)
Rconv = convolve2d(N, k_2D, mode='valid')
matshow(Rconv)
print(Rconv)

### 2. Matrix multiplication

One can write the 2d convolution as a matrix product of a circulant matrix, formed by the convolution kernel, and the vectorized input matrix (e.g. image).

In [ ]:
from scipy.linalg import circulant
k_prime = np.zeros(np.product(S))
k_prime[:len(k)] = k

n = np.reshape(N, [-1, 1])
K = circulant(k_prime).T
#matshow(K, "$circ(k')$")

Rmult_vert = K.dot(n)
Rmult = Rmult_vert.reshape(S)
matshow(Rmult[:, :s-1], '$Rmult$')
print(Rmult)

### 3. Fourier Approach 

Since the convolution matrix is circulant, it is diagonal in Fourier domain. We can use this to solve the system more efficiently 

In [ ]:
from psf2otf import psf2otf

Kf = psf2otf(k[::-1].reshape(1,2), S)
Nf = np.fft.fft2(N)

Rf = np.multiply(Kf, Nf)
Rfourier = np.real(np.fft.ifft2(Rf))

matshow(Rfourier[:, :s-1], '$Rfourier$')
print(Rfourier)

## B. Convex optimization solution

In [ ]:
# Read the NIR image to be deblurred, and the RGB guide
N_b = plt.imread('../input/nir_blurry.tiff');
RGB = plt.imread('../input/rgb.tiff');

# For simplicity we only use the color luminance
Y = np.mean(RGB,axis=2);
# Rescale
Y = Y.astype(np.float)/np.max(Y);
N_b = N_b.astype(np.float)/np.max(N_b);

plt.figure()
plt.subplot(1, 2, 1)
plt.imshow(N_b)
plt.title('Out-of-focus NIR image');
plt.subplot(1, 2, 2)
plt.imshow(RGB)
plt.title('Color guide');
plt.show()

In [ ]:
from scipy.signal import convolve2d
from scipy.signal import gaussian
from math import ceil
from time import time

# We run a blur estimation algorithm, with the strong assumption of a
# constant blur across the image, and that the blur is Gaussian
# We obtain an estimate of sigma = 0.394

#### blur kernel ####

sigma = 0.394;
bsize = ceil(2*3*sigma+1)
b_one = gaussian(bsize, sigma, sym=True)
b = np.outer(b_one, b_one)

# make sure all elements sum up to 1 (like in matlabs fspecial('Gaussian') output)
b = b/np.sum(b)

lamda = 1.0;
eps = 2.2204e-16 # like in MATLAB

#### gradient kernels ####
f1 = np.array([-1, 1]).reshape([1, -1]);
f2 = f1.T;

#### color guides ####

y1 = convolve2d(Y, f1, 'same');
y2 = convolve2d(Y, f2, 'same');

# Fourier domain optimization solution:
start_time = time()

f1F = psf2otf(f1, Y.shape);
f2F = psf2otf(f2, Y.shape);

y1F = psf2otf(y1, Y.shape);
y2F = psf2otf(y2, Y.shape);

bF   = psf2otf(b, Y.shape);
N_bF = psf2otf(N_b, Y.shape);

# EQ (15)
I_x = np.multiply(np.conj(f1F),y1F) +  np.multiply(np.conj(f2F), y2F) + np.multiply(np.conj(bF), N_bF)
C  = lamda * (np.abs(f1F)**2 + np.abs(f2F)**2 + np.abs(bF)**2) + eps

NF = np.divide(I_x, C);
N = np.abs(np.fft.fftshift(np.fft.ifft2(NF)));

end_time = time()
timeTotal = end_time - start_time
print('Donce with optimization after {:2.2f}s'.format(timeTotal))

In [ ]:
# Comparing results

plt.figure()
plt.subplot(1,2,1)
plt.imshow(N_b)
plt.title('Out-of-focus NIR image')
plt.subplot(1,2,2)
plt.imshow(N)
plt.title('Deblurred NIR image')

plt.figure()
plt.subplot(1,2,1)
plt.imshow(N_b[500:900, 650:1200])
plt.title('Out-of-focus CROP');

plt.subplot(1,2,2)
plt.imshow(N[500:900, 650:1200])
plt.title('Deblurred CROP');